<a href="https://colab.research.google.com/github/arcankc/fcc_sms_text_classification/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
    !pip install tf-nightly
except Exception:
    pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers   # ✅ EKLİ
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

# Load data
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

# Map labels to binary
train_df['label_num'] = train_df['label'].map({'ham': 0, 'spam': 1})
test_df['label_num'] = test_df['label'].map({'ham': 0, 'spam': 1})

# Tokenize texts
from tensorflow.keras.layers import TextVectorization

# Parameters
max_features = 10000
sequence_length = 100

# Text Vectorization layer
vectorizer = TextVectorization(max_tokens=max_features, output_mode='int', output_sequence_length=sequence_length)
vectorizer.adapt(train_df['message'].values)

# Create datasets
train_ds = tf.data.Dataset.from_tensor_slices((
    train_df['message'].values,
    train_df['label_num'].values
))

test_ds = tf.data.Dataset.from_tensor_slices((
    test_df['message'].values,
    test_df['label_num'].values
))

# Batch + prefetch
batch_size = 32
train_ds = train_ds.shuffle(10000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Build model
model = tf.keras.Sequential([
    vectorizer,
    layers.Embedding(max_features + 1, 16),
    layers.GlobalAveragePooling1D(),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train → EPOCHS = 30 🚀
history = model.fit(train_ds, validation_data=test_ds, epochs=30)

# Function to predict messages
def predict_message(pred_text):
    pred_input = tf.convert_to_tensor([pred_text])
    prediction = model.predict(pred_input)[0][0]
    label = 'spam' if prediction > 0.5 else 'ham'
    return [float(prediction), label]

# Example
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)

# TEST CELL 🚀
def test_predictions():
    test_messages = ["how are you doing today",
                     "sale today! to stop texts call 98912460324",
                     "i dont want to go. can we try it a different day? available sat",
                     "our new mobile video service is live. just install on your phone to start watching.",
                     "you have won £1000 cash! call to claim your prize.",
                     "i'll bring it tomorrow. don't forget the milk.",
                     "wow, is your arm alright. that happened to me one time too"
                    ]

    test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
    passed = True

    for msg, ans in zip(test_messages, test_answers):
        prediction = predict_message(msg)
        if prediction[1] != ans:
            passed = False

    if passed:
        print("You passed the challenge. Great job!")
    else:
        print("You haven't passed yet. Keep trying!")

# Run test
test_predictions()
